In [1]:
import pandas as pd
import numpy as np

import json
import ast

#### Cleaning Sales Dataframe

In [2]:
sales = pd.read_csv('./data/sales_raw.csv', index_col=0).drop(columns=['source', 'transactionHash', 'blockchainInfo', 'id', 'buyerOrderHash', 'sellerOrderHash', '@type', 'type'])
sales.head(3)

,date,nft,payment,buyer,seller,price,priceUsd,amountUsd
0,2022-02-19T05:46:20Z,"{'type': {'@type': 'ERC721', 'contract': 'ETHE...","{'type': {'@type': 'ETH', 'blockchain': 'ETHER...",ETHEREUM:0x6365398036d4ef045254fce2e525ac59760...,ETHEREUM:0xe6fe14a30631b22d3f78d9ec7869eb62d6e...,3.30,8370.462853,8370.462853
1,2022-02-24T02:15:15Z,"{'type': {'@type': 'ERC721', 'contract': 'ETHE...","{'type': {'@type': 'ETH', 'blockchain': 'ETHER...",ETHEREUM:0xcb4652bb0396880c3b2609d5a37e19df731...,ETHEREUM:0x6365398036d4ef045254fce2e525ac59760...,3.75,9653.748962,9653.748962
2,2022-02-07T06:38:47Z,"{'type': {'@type': 'ERC721', 'contract': 'ETHE...","{'type': {'@type': 'ETH', 'blockchain': 'ETHER...",ETHEREUM:0xc441eb1f41b06826db885e81486b086294a...,ETHEREUM:0xdb6cfff6c0955676bfa8ce795d5beffbab7...,6.90,17423.477522,17423.477522


In [3]:
#extracting token_id from 'nft' column

def token_grabber(s):
    d = ast.literal_eval(s)
    return int(d['type']['tokenId'])

sales.nft = sales.nft.apply(token_grabber)

In [4]:
#changing date to pandas datetime format

sales.date = pd.to_datetime(sales.date)
type(sales.date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [5]:
#extracting payment method from payment:

def pmtmethod_grabber(s):
    d = ast.literal_eval(s)
    return d['type']['@type']

sales.payment = sales.payment.apply(pmtmethod_grabber)

In [6]:
#extracting buyer address from buyer

def address_grabber(s):
    address = s.split(':')[-1]
    return address

sales.buyer = sales.buyer.apply(address_grabber)
sales.seller = sales.seller.apply(address_grabber)

In [7]:
sales.head()

,date,nft,payment,buyer,seller,price,priceUsd,amountUsd
0,2022-02-19 05:46:20+00:00,37,ETH,0x6365398036d4ef045254fce2e525ac59760b186b,0xe6fe14a30631b22d3f78d9ec7869eb62d6e73daa,3.30,8370.462853,8370.462853
1,2022-02-24 02:15:15+00:00,37,ETH,0xcb4652bb0396880c3b2609d5a37e19df73180ede,0x6365398036d4ef045254fce2e525ac59760b186b,3.75,9653.748962,9653.748962
2,2022-02-07 06:38:47+00:00,69,ETH,0xc441eb1f41b06826db885e81486b086294a925fd,0xdb6cfff6c0955676bfa8ce795d5beffbab793298,6.90,17423.477522,17423.477522
3,2022-02-01 03:59:04+00:00,140,ETH,0x8279648470eb92cbcd00ceb8ca30c2adfac20740,0xdb6cfff6c0955676bfa8ce795d5beffbab793298,10.69,32893.590421,32893.590421
4,2022-02-10 20:54:29+00:00,140,ETH,0x0c7598889d8ee611e94566d5cac5cc7c62f516da,0xc55db9017f9fa92f514050fb64439a2cac4ec458,12.00,30301.700037,30301.700037


In [8]:
#Checking for difference in priceUsd and amountUsd

sales.loc[sales.priceUsd != sales.amountUsd]

,date,nft,payment,buyer,seller,price,priceUsd,amountUsd


In [9]:
#dropping redundant columns

sales = sales.drop(columns='amountUsd')

In [10]:
#final check

sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19784 entries, 0 to 19783
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   date      19784 non-null  datetime64[ns, UTC]
 1   nft       19784 non-null  int64              
 2   payment   19784 non-null  object             
 3   buyer     19784 non-null  object             
 4   seller    19784 non-null  object             
 5   price     19784 non-null  float64            
 6   priceUsd  19784 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(3)
memory usage: 1.7+ MB


In [11]:
sales.to_csv('./data/clean_sales_data.csv', index=False)